## Introduction

For our project We are analyzing the dataset of all flights that departed from the three main New York City airports in 2023. For this project We will be analyzing all the features of the dataset and from the auxiliary ones. For the regression task our target variable is 'arr_delay' and for classification task we have chosen flight cancellation prediction.

## Data Cleaning

Overview of datasets:
* airlines.csv
    * `carrier` -- Two-letter airline code.
    * `name` -- Full name of the airline.

* airports.csv
    * `faa` -- FAA code of the airport (e.g., JFK for John F. Kennedy).
    * `name` -- Name of the airport.
    * `lat` -- Latitude of the airport (decimal format).
    * `lon` -- Longitude of the airport (decimal format).
    * `alt` -- Altitude of the airport in feet.
    * `tz` -- Time zone offset (UTC offset). Can be `null`
    * `dst` -- Daylight saving time indicator (`A` = Active, etc.). Can be `null` but it is `null` iff `tz` is `null`
    * `tzone` -- IANA time zone name (e.g., America/New_York). Can be `null` but it is `null` in superset of cases when `tz` is null, therefore can be filled sometimes with already existing values

* flights.csv
    * `year` -- Year of the flight. Always `2023`, therefore dropped
    * `month` -- Month of the flight.
    * `day` -- Day of the flight.
    * `dep_time` -- Actual departure time (local, in military format). Can be `null`
    * `sched_dep_time` -- Scheduled departure time (local, in military format).
    * `dep_delay` -- Departure delay in minutes (negative if early). Can be `null` iff `dep_time` is null
    * `arr_time` -- Actual arrival time (local, in military format). Can be `null` in superset of cases when `dep_delay` is `null`. If it is `null` we assume that flight was cancelled.
    * `sched_arr_time` -- Scheduled arrival time (local, in military format).
    * `arr_delay` -- Arrival delay in minutes (negative if early). Can be `null` in superset of cases when `arr_time` is `null`. Can be filled in as difference between `sched_arr_time` and `arr_time`
    * `carrier` -- Two-letter airline code (links to `airlines.csv`).
    * `flight` -- Flight number.
    * `tailnum` -- Aircraft tail number (links to `planes.csv`). Cna be `null` but number is small => can be dropped.
    * `origin` -- Origin airport code (links to `airports.csv`).
    * `dest` -- Destination airport code (links to `airports.csv`).
    * `air_time` -- Total air time in minutes. Can be `null` iff `arr_delay` is null but we decided not to use this column => can be ignored.
    * `distance` -- Distance of the flight in miles.
    * `hour` -- Scheduled departure hour (derived from `sched_dep_time`).
    * `minute` -- Scheduled departure minute (derived from `sched_dep_time`).
    * `time_hour` -- Rounded time to the nearest hour (useful for joining with weather data).

* planes.csv
    * `tailnum` -- Unique aircraft identifier (matches with `flights.tailnum`).
    * `year` -- Year the plane was manufactured. Can be `null` but it is quite rare case, therefore can be dropped
    * `type` -- Aircraft type (e.g., "Fixed wing multi engine").
    * `manufacturer` -- Manufacturer of the aircraft (e.g., Boeing, Airbus).
    * `model` -- Model of the aircraft (e.g., "A320").
    * `engines` -- Number of engines on the aircraft.
    * `seats` -- Number of passenger seats on the plane. Always `0`, therefore dropped
    * `speed` -- Typical cruise speed (if available; may be NaN).
    * `engine` -- Type of engine (e.g., "Turbo-fan").

* weather.csv
    * `origin` -- Airport code where the weather data is recorded (links to `airports.faa`).
    * `year` -- Year of the weather record.
    * `month` -- Month of the weather record.
    * `day` -- Day of the weather record.
    * `hour` -- Hour of the weather observation (local).
    * `temp` -- Temperature in degrees Fahrenheit. Can be `null` in superset of cases when `precip` is `null`
    * `dewp` -- Dewpoint temperature in degrees Fahrenheit. Can be `null` iff `temp` is `null`
    * `humid` -- Relative humidity (%). Can be `null` iff `temp` is `null`
    * `wind_dir` -- Wind direction in degrees (0° = North, 90° = East, etc.).
    * `wind_speed` -- Wind speed in mph.
    * `wind_gust` -- Wind gust speed in mph (if available).
    * `precip` -- Precipitation in inches. Can be `null`.
    * `pressure` -- Atmospheric pressure in millibars (if available). Can be `null` in superset of cases when `temp` is `null`
    * `visib` -- Visibility in miles.
    * `time_hour` -- Time of the observation rounded to the nearest hour (links to `flights.time_hour`).



### Imports

In [ ]:
!pip install -r requirements.txt

In [ ]:
import numpy as np

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

### Load datasets

In [ ]:
def read_csv(filename):
    df = pd.read_csv(
        filename,
        header=0,                     # Use the first row as the header
        delimiter=';',                # Use semicolon as the main delimiter
        decimal=',',                  # Specify that commas are used as decimals
        quotechar='"',                # Handle quotes around strings
        skipinitialspace=True,        # Skip spaces after delimiters
    )
    df.set_index(df.columns[0], inplace=True)
    return df

airlines = read_csv('airlines.csv')
airports = read_csv('airports.csv')
planes = read_csv('planes.csv')
flights = read_csv('flights.csv')
weather = read_csv('weather.csv')

Some columns have float type instead of int, we need to change it

In [ ]:
planes['year'] = planes['year'].astype('Int64')
flights['dep_time'] = flights['dep_time'].astype('Int64')
flights['dep_delay'] = flights['dep_delay'].astype('Int64')
flights['arr_time'] = flights['arr_time'].astype('Int64')
flights['arr_delay'] = flights['arr_delay'].astype('Int64')

In [ ]:
def check_for_nulls(df):
    res = {}
    for c in df.columns:
        if df[df[c].isnull()].shape[0] != 0:
            res[c] = df[df[c].isnull()].shape[0]
    return res

Let's analyze nulls in datasets

In [ ]:
print("Nullable columns in airlines:", check_for_nulls(airlines))
print("Nullable columns in airports:", check_for_nulls(airports))
print("Nullable columns in planes:", check_for_nulls(planes))
print("Nullable columns in flights:", check_for_nulls(flights))
print("Nullable columns in weather:", check_for_nulls(weather))

After careful analysis we found dependencies that were mentioned in data overview

### Clean Datasets

We have observed, that for some flights even though the scheduled departure and arrival times are present, the actual departure and arrival times are absent (i.e., NaN). We have deduced, that this can be interpreted as a cancellation of said flight. We have checked this hypothesis with publicly available information, and it was confirmed.

We need to clean each dataset from nulls according to our expectations of data

In [ ]:
### Clean Datasets

In [ ]:
def clear_airports(df): # fill `tzone` if it can be filled, drop otherwise. If `tz` is unknown, drop
    df = df.dropna(subset=['tz', 'dst'])

    for idx, row in df[df['tzone'].isnull()].iterrows():
        matching_row = df[(df['tz'] == row['tz']) & pd.notnull(df['tzone'])]
        if not matching_row.empty:
            df.at[idx, 'tzone'] = matching_row['tzone'].iloc[0]
        else:
            df = df.drop(idx)
    return df

In [ ]:
def clear_planes(df): # drop all planes if `year` is unknown
    return df.dropna()

In [ ]:
def clear_flights(df): # drop if `tailnum` is unknown, fix military format of time, fill air_time where needed
    df = df.dropna(subset=['tailnum'])

    for idx, row in df.iterrows():
        df.at[idx, 'dep_time'] = df.at[idx, 'dep_time'] % 100 + (df.at[idx, 'dep_time'] // 100) * 60
        df.at[idx, 'arr_time'] = df.at[idx, 'arr_time'] % 100 + (df.at[idx, 'arr_time'] // 100) * 60
        df.at[idx, 'sched_dep_time'] = df.at[idx, 'sched_dep_time'] % 100 + (df.at[idx, 'sched_dep_time'] // 100) * 60
        df.at[idx, 'sched_arr_time'] = df.at[idx, 'sched_arr_time'] % 100 + (df.at[idx, 'sched_arr_time'] // 100) * 60

    for idx, row in df[df['air_time'].isnull()].iterrows():
        # df.at[idx, 'arr_delay'] = df['arr_time'].iloc[0] - df['sched_arr_time'].iloc[0]
        df.at[idx, 'air_time'] = df['arr_time'].iloc[0] - df['dep_time'].iloc[0]
    return df

In [ ]:
def clear_weather(df): # here we ignore all nulls and keep them as is
    return df

In [ ]:
clean_airlines = airlines
clean_airports = clear_airports(airports.copy())
clean_planes = clear_planes(planes.copy())
clean_flights = clear_flights(flights.copy())
clean_weather = clear_weather(weather.copy())

In [ ]:
print("Nullable columns in airlines:", check_for_nulls(clean_airlines))
print("Nullable columns in airports:", check_for_nulls(clean_airports))
print("Nullable columns in planes:", check_for_nulls(clean_planes))
print("Nullable columns in flights:", check_for_nulls(clean_flights))
print("Nullable columns in weather:", check_for_nulls(clean_weather))

In [ ]:
print("Airports old vs new:", airports.shape, clean_airports.shape)
print("Planes old vs new:", planes.shape, clean_planes.shape)
print("Flights old vs new:", flights.shape, clean_flights.shape)
print("Weather old vs new:", weather.shape, clean_weather.shape)

### Merge datasets

We need to merge all data together: add info about airports, planes and weather in the origin airport

In [ ]:
# merge flights with info about origin airport
flights_with_origin_info = pd.merge(
    clean_flights,
    clean_airports,
    how='left',
    left_on='origin',
    right_on='faa',
    suffixes=('', '_origin')
)

# merge obtained dataset with info about destination airport
flights_with_origin_and_dest = pd.merge(
    flights_with_origin_info,
    clean_airports,
    how='inner',
    left_on='dest',
    right_on='faa',
    suffixes=('_origin', '_dest')
)

# merge obtained dataset with info about plane
flights_with_origin_dest_and_planes = pd.merge(
    flights_with_origin_and_dest,
    clean_planes,
    how='inner',
    left_on=['tailnum'],
    right_on=['tailnum'],
    suffixes=('', '_plane')
)

# merge obtained dataset with info about weather at the departure time
merged_df = pd.merge(
    flights_with_origin_dest_and_planes,
    clean_weather,
    how='left',
    left_on=['origin', 'year', 'month', 'day', 'hour'],
    right_on=['origin', 'year', 'month', 'day', 'hour'],
    suffixes=('', '_weather')
)

In [ ]:
merged_df['year_plane'] = merged_df['year'].astype(int)
merged_df['is_cancelled'] = merged_df['arr_time'].isnull()

# drop columns that are not needed for classification and regression
merged_df = merged_df.drop(['arr_time', 'flight', 'tailnum', 'air_time', 'time_hour', 'faa_origin', 'name_origin', 'lat_origin', 'lon_origin', 'alt_origin', 'tz_origin', 'dst_origin', 'tzone_origin', 'faa_dest', 'name_dest', 'dst_dest', 'tzone_dest', 'time_hour_weather', 'year', 'speed', 'hour', 'minute'], axis=1)

In [ ]:
merged_df.to_csv("merged_df.csv", index=False)

In [ ]:
# split into 3 datasets depending on origin airport
ewr_df = merged_df[merged_df['origin'] == 'EWR'].drop('origin', axis=1)
jfk_df = merged_df[merged_df['origin'] == 'JFK'].drop('origin', axis=1)
lga_df = merged_df[merged_df['origin'] == 'LGA'].drop('origin', axis=1)

In [ ]:
# process each dataset and create datasets for both classification and regression task
ewr_delay_df = ewr_df.drop('is_cancelled', axis=1).dropna(subset=['arr_delay', 'dep_delay', 'dep_time'])
ewr_cancel_df = ewr_df.drop(['dep_delay', 'arr_delay'], axis=1)
jfk_delay_df = jfk_df.drop('is_cancelled', axis=1).dropna(subset=['arr_delay', 'dep_delay', 'dep_time'])
jfk_cancel_df = jfk_df.drop(['dep_delay', 'arr_delay'], axis=1)
lga_delay_df = lga_df.drop('is_cancelled', axis=1).dropna(subset=['arr_delay', 'dep_delay', 'dep_time'])
lga_cancel_df = lga_df.drop(['dep_delay', 'arr_delay'], axis=1)

In [ ]:
ewr_delay_df.to_csv("ewr_delay.csv", index=False)
jfk_delay_df.to_csv("jfk_delay.csv", index=False)
lga_delay_df.to_csv("lga_delay.csv", index=False)
ewr_cancel_df.to_csv("ewr_cancel.csv", index=False)
jfk_cancel_df.to_csv("jfk_cancel.csv", index=False)
lga_cancel_df.to_csv("lga_cancel.csv", index=False)

## Exporatory Data Analysis
### Delays

First, let's take a general look over the data distribution. To start with, how many flights are done each month and how delayed flights are distributed over them.

In [ ]:
import seaborn as sns

In [ ]:
monthly_flights = merged_df.groupby('month').size()

plt.figure(figsize=(10, 6))
monthly_flights.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Flights per Month', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Delayed Flights', fontsize=12)
plt.xticks(ticks=range(12), labels=[f'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
flights_delayed = merged_df[(merged_df['arr_delay'] > 15) | (merged_df['dep_delay'] > 15)]

monthly_delays = flights_delayed.groupby('month').size()

plt.figure(figsize=(10, 6))
monthly_delays.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Flights Delayed > 15 Minutes per Month', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Delayed Flights', fontsize=12)
plt.xticks(ticks=range(12), labels=[f'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

From the plot above we may try to derive some conclusions:
- It is clear that the most delays occur in the peak summer months June and July, which may be due to overall increased air traffic, congestions of airports and weather conditions
- Similarly, autumn months, when the travels are less frequent, are less affected by delays

To analyse this further, we can try to look into differences between arr_delay and dep_delay:

In [ ]:
flights_delayed = merged_df[(merged_df['dep_delay'] > 15)]

monthly_delays = flights_delayed.groupby('month').size()

plt.figure(figsize=(10, 6))
monthly_delays.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Flights Departed > 15 Minutes than expected per Month', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Delayed Flights', fontsize=12)
plt.xticks(ticks=range(12), labels=[f'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
flights_delayed = merged_df[(merged_df['arr_delay'] > 15)]

monthly_delays = flights_delayed.groupby('month').size()

plt.figure(figsize=(10, 6))
monthly_delays.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Flights Arrived > 15 Minutes than expected per Month', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Delayed Flights', fontsize=12)
plt.xticks(ticks=range(12), labels=[f'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

The plots show that in some months there is a tendency in delayed departures, but not at the identified peak months. Let's see how delayed arrivals are distributed over the source airports:

In [ ]:
flights_delayed = merged_df[(merged_df['arr_delay'] > 15) | (merged_df['dep_delay'] > 15)]

delays_by_destination = flights_delayed.groupby('origin').size().sort_values(ascending=False)

plt.figure(figsize=(14, 7))
delays_by_destination.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Delayed Flights (> 15 Minutes) by Origin Airport', fontsize=16)
plt.xlabel('Destination Airport', fontsize=12)
plt.ylabel('Number of Delayed Flights', fontsize=12)
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

And what are mean delays in them:

In [ ]:
flights_delayed.groupby('origin')[['dep_delay', 'arr_delay']].mean().sort_values('arr_delay', ascending=False)

There is no significant difference between the origin airports in terms of delays, let's investigate the destination airports in the similar way:

In [ ]:
flights_delayed = merged_df[(merged_df['arr_delay'] > 15) | (merged_df['dep_delay'] > 15)]

delays_by_destination = flights_delayed.groupby('dest').size().sort_values(ascending=False)
delays_by_destination = delays_by_destination[delays_by_destination >= 500]

# Step 3: Plot the result
plt.figure(figsize=(14, 7))
delays_by_destination.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Delayed Flights (> 15 Minutes) by Destination Airport (>500 delays over year)', fontsize=16)
plt.xlabel('Destination Airport', fontsize=12)
plt.ylabel('Number of Delayed Flights', fontsize=12)
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

The above plot does not contain destination airpots with less than 500 delayed flights over the year (for visual purposes). We can clearly observe a skewed distribution with several airports constituting to the most of the delays. To indicate it more formally, we can investigate cumulative percentage over Pareto threshold:

In [ ]:
delays_by_destination = flights_delayed.groupby('dest').size()
delays_by_destination = delays_by_destination[delays_by_destination >= 100].sort_values(ascending=False)

cumulative_delays = delays_by_destination.cumsum()
cumulative_percentage = 100 * cumulative_delays / cumulative_delays.iloc[-1]

plt.figure(figsize=(12, 6))
plt.plot(cumulative_percentage, marker='o', color='skyblue', label='Cumulative Percentage')
plt.axhline(y=80, color='r', linestyle='--', label='80% Threshold')

plt.title('Cumulative Percentage of Delays by Destination Airport', fontsize=16)
plt.xlabel('Destination Airport (sorted by delay frequency)', fontsize=12)
plt.ylabel('Cumulative Percentage (%)', fontsize=12)
plt.xticks(ticks=np.arange(len(cumulative_percentage)), labels=cumulative_percentage.index, rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

pareto_threshold = (cumulative_percentage <= 80).sum()
print(f"Number of destinations accounting for 80% of delays: {pareto_threshold} out of {len(delays_by_destination)}")

With this, we see that around 20-30% of the airports (on the left) contribute most to the delays: after reaching 80% the curve significantly flattens. 

Similarly, by observing amount of flights for each hour of the day versus delayed flights we can see that evening hours contribute more to the delays:

In [ ]:
flights_delayed = merged_df[(merged_df['arr_delay'] > 15) | (merged_df['dep_delay'] > 15)]

flights_delayed['dep_hour'] = flights_delayed['dep_time'] // 60

hourly_delays = flights_delayed.groupby('dep_hour').size()

plt.figure(figsize=(10, 6))
hourly_delays.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Flights Delayed > 15 Minutes per Hour', fontsize=14)
plt.xlabel('Hour of the Day', fontsize=12)
plt.ylabel('Number of Delayed Flights', fontsize=12)
plt.xticks(ticks=range(24), labels=[f'{hour}:00' for hour in range(24)], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
all_flights = merged_df
all_flights['dep_hour'] = all_flights['dep_time'] // 60

hourly_all_delays = all_flights.groupby('dep_hour').size()

plt.figure(figsize=(10, 6))
hourly_all_delays.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Flights Delayed > 15 Minutes per Hour', fontsize=14)
plt.xlabel('Hour of the Day', fontsize=12)
plt.ylabel('Number of Delayed Flights', fontsize=12)
plt.xticks(ticks=range(24), labels=[f'{hour}:00' for hour in range(24)], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Which can be for various reasons:
- Operational congestion of accumulating delays and mismanagements throughout the day (thus the increasing of the amount of delays) 
- Less space for adjustments 
- Unequal staffing or external conditions like weather and decreased visibility in the evening hours

The weather data for the dataset mostly lacks precise temp/humidity/etc data. We can try to observe the distribution of existing to understand whether this data can be reliably used in the future investigation:

In [ ]:
airports = ["JFK", "EWR", "LGA"]

for airport in airports:
    filtered_weather = clean_weather[clean_weather['origin'].eq(airport) 
                                     & clean_weather['temp'].notnull() 
                                     & clean_weather['dewp'].notnull() 
                                     & clean_weather['humid'].notnull()]
    
    filtered_weather['date'] = pd.to_datetime(filtered_weather[['year', 'month', 'day']])    
    daily_counts = filtered_weather.groupby('date').size()
    
    plt.figure(figsize=(10, 6))
    daily_counts.plot(kind='line', title='Number of Weather per Day for Selected Airports')
    plt.xlabel('Date')
    plt.ylabel('Number of Rows')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.show()

It is clear that only a few dates during different periods have almost complete weather coverage in all three airports, which is not enough to rely on weather data when solving the problem. 

Let's observe if the delays depend on a particular carrier:

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=merged_df[merged_df['arr_delay'] < 100], x='carrier', y='arr_delay', palette="Set2")

plt.title("Arrival Delays by Carrier")
plt.xlabel("Carrier")
plt.ylabel("Delay (minutes)")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=merged_df[merged_df['dep_delay'] < 100], x='carrier', y='dep_delay', palette="Set2", color="orange")

plt.title("Departure Delays by Carrier")
plt.xlabel("Carrier")
plt.ylabel("Delay (minutes)")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

There is a clear observable dependency between carriers and delays (we cropped the data not to include delays over 100 minutes for visual representation).

Finally, seeing a correlation matrix for delays can be beneficial to see if there are any more dependencies worth investigating.

In [ ]:
corr_df = merged_df.select_dtypes(include=np.number).corr()

plt.figure(figsize=(14, 8))
sns.heatmap(data=corr_df[['arr_delay', 'dep_delay']], annot=True, cmap='YlGnBu')

plt.show()

Apart from weather conditions, which we considered unreliable, there is no meaningful correlation with other variables.

## Cancellations
We performed similar analysis for cancellations.

In [ ]:
flights_canceled = merged_df[merged_df['is_cancelled']]
monthly_cancellations = flights_canceled.groupby('month').size()

plt.figure(figsize=(10, 6))
monthly_cancellations.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Flights Cancelled per Month', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Cancelled Flights', fontsize=12)
plt.xticks(ticks=range(12), labels=[f'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### Regression Task

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

In [ ]:
def turn_into_regression_dataset(dataframe):
    target_column = 'arr_delay'
    additional_columns = ['dep_delay', 'temp', 'dewp', 'humid', 'precip', 'pressure']
    dataframe = dataframe.drop(additional_columns, axis=1)
    dataframe = dataframe.dropna()
    datafram = dataframe.reset_index(drop=True)
    X = dataframe.drop([target_column], axis=1)
    y = dataframe[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=566)

    categorical_features = X.select_dtypes(include=['object', 'category']).columns
    numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numerical_features),  # Keep numerical features as is
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # One-hot encode categorical features
            ]
        )
    
    X_train_processed = preprocessor.fit_transform(X_train).toarray()
    X_test_processed = preprocessor.transform(X_test).toarray()
    return (X_train_processed, y_train, X_test_processed, y_test)

In [ ]:
def SGDRegression(x_train, y_train, x_test, y_test):
    model = SGDRegressor(
                random_state=566
    )
    training_losses = []
    test_losses = []

    epochs = 10
    n_iterations = 10
    batch_size = len(x_train) // n_iterations
    for epoch in range(1, epochs + 1):
        for i in range(1, n_iterations + 1):
            X_batch = x_train[(i - 1) * batch_size : i * batch_size]
            y_batch = y_train[(i - 1) * batch_size : i * batch_size]
    
            model.partial_fit(X_batch, y_batch)
    
            y_train_pred = model.predict(X_batch)
            y_test_pred = model.predict(x_test)
    
            training_loss = np.mean((y_batch - y_train_pred) ** 2)
            test_loss = np.mean((y_test - y_test_pred) ** 2)
    
            training_losses.append(training_loss)
            test_losses.append(test_loss)
            print(f"Epoch: {epoch}, Iteration {i}: Training Loss = {training_loss:.4f}, Test Loss = {test_loss:.4f}")
    return training_losses, test_losses

def regressionMLP(x_train, y_train, x_test, y_test):
    model = MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation='relu',
        solver='adam',
        max_iter=500,
        random_state=566
    )
    training_losses = []
    test_losses = []

    epochs = 10
    n_iterations = 10
    batch_size = len(x_train) // n_iterations
    for epoch in range(1, epochs + 1):
        for i in range(1, n_iterations + 1):
            X_batch = x_train[(i - 1) * batch_size : i * batch_size]
            y_batch = y_train[(i - 1) * batch_size : i * batch_size]
    
            model.partial_fit(X_batch, y_batch)
    
            y_train_pred = model.predict(X_batch)
            y_test_pred = model.predict(x_test)
    
            training_loss = np.mean((y_batch - y_train_pred) ** 2)
            test_loss = np.mean((y_test - y_test_pred) ** 2)
    
            training_losses.append(training_loss)
            test_losses.append(test_loss)
            print(f"Epoch: {epoch}, Iteration {i}: Training Loss = {training_loss:.4f}, Test Loss = {test_loss:.4f}")
    return training_losses, test_losses

In [ ]:
def plot_losses(model, training_losses, test_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(training_losses) + 1), training_losses, label=f'{model}: Training Loss', marker='o')
    plt.plot(range(1, len(test_losses) + 1), test_losses, label=f'{model}: Test Loss', marker='o')
    plt.xlabel('Iteration')
    plt.ylabel('Mean Squared Error')
    plt.title('Training and Test Loss Over Iterations')
    plt.legend()
    plt.grid()
    plt.show()

#### EWR Airport

In [ ]:
x_train, y_train, x_test, y_test = turn_into_regression_dataset(ewr_delay_df)
SGDLoss = SGDRegression(x_train, y_train, x_test, y_test)
plot_losses("SGD", *SGDLoss)

In [ ]:
mlp_losses = regressionMLP(x_train, y_train, x_test, y_test)
plot_losses("MLP regression", *mlp_losses)

#### JFK Airport

In [ ]:
x_train, y_train, x_test, y_test = turn_into_regression_dataset(jfk_delay_df)
SGDLoss = SGDRegression(x_train, y_train, x_test, y_test)
plot_losses("SGD", *SGDLoss)

In [ ]:
mlp_losses = regressionMLP(x_train, y_train, x_test, y_test)
plot_losses("MLP regression", *mlp_losses)

#### LGA Airport

In [ ]:
x_train, y_train, x_test, y_test = turn_into_regression_dataset(lga_delay_df)
SGDLoss = SGDRegression(x_train, y_train, x_test, y_test)
plot_losses("SGD", *SGDLoss)

In [ ]:
mlp_losses = regressionMLP(x_train, y_train, x_test, y_test)
plot_losses("MLP regression", *mlp_losses)

### Classification task

In [ ]:
def turn_into_classification_dataset(dataframe):
    target_column = 'is_cancelled'
    additional_columns = ['temp', 'dewp', 'humid', 'precip', 'pressure']
    dataframe = dataframe.drop(additional_columns, axis=1)
    dataframe = dataframe.dropna()
    datafram = dataframe.reset_index(drop=True)
    X = dataframe.drop([target_column], axis=1)
    y = dataframe[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=566)

    categorical_features = X.select_dtypes(include=['object', 'category']).columns
    numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numerical_features),  # Keep numerical features as is
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # One-hot encode categorical features
            ]
        )
    
    X_train_processed = preprocessor.fit_transform(X_train).toarray()
    X_test_processed = preprocessor.transform(X_test).toarray()
    return (X_train_processed, y_train, X_test_processed, y_test)

In [ ]:
def logReg(x_train, y_train, x_test, y_test):
    model = LogisticRegression(
                max_iter=500,
                random_state=566,
                warm_start=True
            )
    training_losses = []
    test_losses = []

    epochs = 10
    n_iterations = 10
    batch_size = len(x_train) // n_iterations
    for i in range(1, n_iterations + 1):
        X_batch = x_train[(i - 1) * batch_size : i * batch_size]
        y_batch = y_train[(i - 1) * batch_size : i * batch_size]
    
        model.fit(X_batch, y_batch)
    
        y_train_proba = model.predict_proba(X_batch)[:, 1]
        y_test_proba = model.predict_proba(x_test)[:, 1]
            
        training_loss = log_loss(y_batch, y_train_proba)
        test_loss = log_loss(y_test, y_test_proba)
    
        training_losses.append(training_loss)
        test_losses.append(test_loss)
        print(f"Iteration {i}: Training Loss = {training_loss:.4f}, Test Loss = {test_loss:.4f}")
    return training_losses, test_losses, model.predict(x_test)

def randForClass(x_train, y_train, x_test, y_test):
    model = RandomForestClassifier(
        n_estimators=100,         
        max_depth=10,    
        random_state=566,     
        n_jobs=-1,
        warm_start=True
    )

    training_losses = []
    test_losses = []

    epochs = 10
    n_iterations = 10
    batch_size = len(x_train) // n_iterations
    for i in range(1, n_iterations + 1):
        X_batch = x_train[(i - 1) * batch_size : i * batch_size]
        y_batch = y_train[(i - 1) * batch_size : i * batch_size]
    
        model.fit(X_batch, y_batch)
        model.n_estimators += 10
    
        y_train_proba = model.predict_proba(X_batch)[:, 1]
        y_test_proba = model.predict_proba(x_test)[:, 1]
            
        training_loss = log_loss(y_batch, y_train_proba)
        test_loss = log_loss(y_test, y_test_proba)
    
        training_losses.append(training_loss)
        test_losses.append(test_loss)
        print(f"Iteration {i}: Training Loss = {training_loss:.4f}, Test Loss = {test_loss:.4f}")
    return training_losses, test_losses, model.predict(x_test)

#### EWR Airport

In [ ]:
x_train, y_train, x_test, y_test = turn_into_classification_dataset(ewr_cancel_df)

In [ ]:
LogLosses1, LogLosses2, _ = logReg(x_train, y_train, x_test, y_test)
plot_losses("Logistic Regression for classification", LogLosses1, LogLosses2)

In [ ]:
RandForLosses1, RandForLosses2, _ = randForClass(x_train, y_train, x_test, y_test)
plot_losses("Random Forests for classification", RandForLosses1, RandForLosses2)

#### JFK Airport

In [ ]:
x_train, y_train, x_test, y_test = turn_into_classification_dataset(jfk_cancel_df)

In [ ]:
LogLosses1, LogLosses2, _ = logReg(x_train, y_train, x_test, y_test)
plot_losses("Logistic Regression for classification", LogLosses1, LogLosses2)

In [ ]:
plot_losses("Logistic Regression for classification", LogLosses1, LogLosses2)
RandForLosses1, RandForLosses2, _ = randForClass(x_train, y_train, x_test, y_test)
plot_losses("Random Forests for classification", RandForLosses1, RandForLosses2)

#### LGA Airport

In [ ]:
x_train, y_train, x_test, y_test = turn_into_classification_dataset(jfk_cancel_df)

In [ ]:
LogLosses1, LogLosses2, y_pred = logReg(x_train, y_train, x_test, y_test)
plot_losses("Logistic Regression for classification", LogLosses1, LogLosses2)

In [ ]:
RandForLosses1, RandForLosses2, y_pred2 = randForClass(x_train, y_train, x_test, y_test)
plot_losses("Random Forests for classification", RandForLosses1, RandForLosses2)